# Astrangia poculata transcriptome assembly

This notebook contains the code for the *Astrangia poculata* reference transcriptome assembly. This is a de novo assembly constructed using data from RNAseq (Illumina). The coral colonies used to extract the RNA for this assembly were collected from Fort Wetherhill State Park in Jamestown, RI. Additionally, since *A. poculata* is a facultatively symbiotic coral, the colonies used to generate the assembly were from 10 brown and white individuals (5 brown, 5 white). These colonies we previously used in an experiment where some were exposed to a heat-killed pathogen and others were left unexposed. The final breakdown of individuals used in this assembly was 2 brown control, 2 white control, 3 brown pathogen-exposed, 3 white pathogen-exposed. 

The total input for the assembly was 219,350,638 reads and the final assembly consisted of 158,336 transcripts. The best quality assembly was done using Trinity. Multiple iterations of this assembly were done using different assemblers and methods including short read assemblies (RNAeq - Illumina), long read assemblies (IsoSeq - PacBio), and hybrid assemblies (both short and long read data). We also ran iterations with different number of samples for input and different sample combinations. Below is the code used to generate our highest quality assembly and notes on the data from each step.
<a id="0"></a> <br>
### Contents
#### [1. Installing programs](#1)
#### [2. Adapter and quality trimming](#2) 
#### [3. Symbiont read filtering](#3)
#### [4. Concatenating reads for assembly](#4)
#### [5. Transcriptome assembly](#5)
#### [6. Quality assessment of assembly](#6)
#### [7. Assembly filtering](#7)
#### [8. Annotations](#8)


<a id='1'></a>
### 1. Program installation
For this pipeline you will need cutadapt, BBMap, Bowtie2, Trinity, Salmon, and


<a id='2'></a>
### 2. Adapter and quality trimming
Raw sequence files were quality assessed using FastQC. Each sequence file was then trimmed using cutadapt to trim adapter sequences off the ends of reads, and remove low quality sequences (average Phred score <20), and sequences with N base calls.

The first line of code runs FastQC on all `.fastq` files in the directory where the sequence data is located. The second code runs `cutadapt` in array on all sequence files. The cutadapt code provided is the bash script written to be run in array format on the LEAP2 server at Texas State. 

In [ ]:
fastqc *.fastq.gz

In [ ]:
#!/bin/bash
#SBATCH --job-name=cutadapt
#SBATCH -N 1
#SBATCH -t 6-24:00
#SBATCH --partition=shared
#SBATCH --mem=50G
#SBATCH --array=103-112%10
#SBATCH --mail-type=end
#SBATCH --mail-user=eborbee@txstate.edu
#SBATCH -o cutadapt_%j.out
#SBATCH -e cutadapt_%j.err

cutadapt -a AGATCGGAAGAG -m 25  -q 20 --untrimmed-output EB_${SLURM_ARRAY_TASK_ID}_1_untrimmed.fastq \
--untrimmed-paired-output EB_${SLURM_ARRAY_TASK_ID}_2_untrimmed.fastq -o EB_${SLURM_ARRAY_TASK_ID}_1_trimmed.fastq \
-p EB_${SLURM_ARRAY_TASK_ID}_2_trimmed.fastq EB_${SLURM_ARRAY_TASK_ID}_1.fq EB_${SLURM_ARRAY_TASK_ID}_2.fq

In this cutadapt script, the `-a` provides the 3' adapter sequence, the `-m` is the minimum sequence length, the `-q` is the minimum Phred quality score for each sequence, `untrimmed-output` is the forward reads that did not contain the adapter and therefore were not trimmed, `--untrimmed-paired-output` is the same as the untrimmed output but for the reverse read, `-o` is the output for the forward read, and `-p` is the output for the reverse read. Rather than run multiple commands for each sample, this script is run by designating the sample IDs from the file name in the script header using `#SBATCH --array=[SAMPLE_#s]`. The `%10` at the end of the array designation in the header tells the job manager how many jobs to run simultaneously. So to avoid running too many simultaneously, this number should never be larger than 10. The last step in setting up the array is replacing the sample number portion of the filenames with `${SLURM_ARRAY_TASK_ID}`. 

Since not all reads contained the adapter, the trimmed outputs were concatenated to the untrimmed outputs to avoid data loss. Short and low quality reads remained removed from the sequences in this step. We combined these two files using the `cat` command. Below is the bash script that was submitted on the LEAP2 server at Texas State University using an array as explained above in the cutadapt script.

In [ ]:
#!/bin/bash
#SBATCH --job-name=concatenate
#SBATCH -N 1
#SBATCH -t 6-24:00
#SBATCH --partition=shared
#SBATCH --mem=50G
#SBATCH --array=103-112%10
#SBATCH --mail-type=end
#SBATCH --mail-user=eborbee@txstate.edu
#SBATCH -o concatenate_%j.out
#SBATCH -e concatenate_%j.err

cat EB_${SLURM_ARRAY_TASK_ID}_1_trimmed.fastq EB_${SLURM_ARRAY_TASK_ID}_1_untrimmed.fastq > EB_${SLURM_ARRAY_TASK_ID}_1_cat.fastq

cat EB_${SLURM_ARRAY_TASK_ID}_2_trimmed.fastq EB_${SLURM_ARRAY_TASK_ID}_2_untrimmed.fastq > EB_${SLURM_ARRAY_TASK_ID}_2_cat.fastq

<a id='3'></a>
### 3. Symbiont read filtering
Coral RNA extractions extract both coral host and algal symbiont RNA. So after trimming adapter sequences, we then filtered our reads to remove symbiont sequences from our data. We did this by using `bbsplit` to map reads to the symbiont (*Breviolum psygmophilum*) reference transcriptome and any reads that don't match the symbiont reference are saved into a new file of just coral reads.

The first block of code is the bbsplit script that was used to remove the symbiont reads. Just as with the cutadapt script this script was run in array to remove the symbiont reads from the forward and reverse reads of each sample. The *Breviolum psygmophilum* reference transcriptome that was used for this script was downloaded from Reef Genomics. The paper the transcriptome was pulled from is cited below and the website where the file was downloaded from is also provided. The transcriptome was published prior to the reclassification of *Symbiodinium* so the transcriptome is listed under *Symbiodinium psygmophilum* rather than *Breviolum psygmophilum*.

#### Reference:
Parkinson, J. E., Baumgarten, S., Michell, C. T., Baums, I. B., LaJeunesse, T. C., & Voolstra, C. R. (2016). Gene expression variation resolves species and individual strains among coral-associated dinoflagellates within the genus *Symbiodinium*. *Genome Biology and Evolution*, 8(3), 665-680.

#### *B. psygmophilum* reference transcriptome download link:
http://reefgenomics.org/sitemap.html

In [ ]:
#!/bin/bash
#SBATCH --job-name=bbsplit
#SBATCH -N 1
#SBATCH -t 13-24:00
#SBATCH --partition=himem
#SBATCH --mem=250G
#SBATCH --array=103-112
#SBATCH --mail-type=end
#SBATCH --mail-user=eborbee@txstate.edu
#SBATCH -o bbsplit_%j.out
#SBATCH -e bbsplit_%j.err

~/miniconda3/envs/trinity/bin/bbsplit.sh ref=~/BrevPsygmophilum_transcriptome/psyg_assembly_longest_250.fa \
in1=EB_${SLURM_ARRAY_TASK_ID}_1_cat.fastq in2=EB_${SLURM_ARRAY_TASK_ID}_2_cat.fastq basename=out_%.fa \
refstats=sampleStats.txt outu1=unmatched_${SLURM_ARRAY_TASK_ID}_reads1.fa \
outu2=unmatched_${SLURM_ARRAY_TASK_ID}_reads2.fa

<a id='4'></a>
### 4. Concatenating reads for assembly
The output from the bbsplit script is a series of files for each sample titled `unmatched_[SAMPLE_ID]_reads1.fa` and `unmatched_[SAMPLE_ID]_reads2.fa`. The next step is to concatenate the forward and reverse reads from all samples to generate two files, one containing all the forward reads and one containing all the reverse reads. 

In [ ]:
#!/bin/bash
#SBATCH --job-name=concatenate
#SBATCH -N 1
#SBATCH -t 6-24:00
#SBATCH --partition=shared
#SBATCH --mem=50G
#SBATCH --mail-type=end
#SBATCH --mail-user=eborbee@txstate.edu
#SBATCH -o concatenate_%j.out
#SBATCH -e concatenate_%j.err

cat unmatched_103_reads1.fa unmatched_104_reads1.fa unmatched_105_reads1.fa unmatched_106_reads1.fa \
unmatched_107_reads1.fa unmatched_108_reads1.fa unmatched_109_reads1.fa unmatched_110_reads1.fa \
unmatched_111_reads1.fa unmatched_112_reads1.fa > unmatched_reads1.fa

cat unmatched_103_reads2.fa unmatched_104_reads2.fa unmatched_105_reads2.fa unmatched_106_reads2.fa \
unmatched_107_reads2.fa unmatched_108_reads2.fa unmatched_109_reads2.fa unmatched_110_reads2.fa \
unmatched_111_reads2.fa unmatched_112_reads2.fa > unmatched_reads2.fa

<a id='4'></a>
### 4. Transcriptome assembly
The next step is the transcriptome assembly using Trinity. Trinity uses three phases to construct a de novo transcriptome assembly. For more information about how the Trinity assembly works, the link to their wiki is provided below.

#### Trinity wiki:
https://github.com/trinityrnaseq/trinityrnaseq/wiki

In [ ]:
#!/bin/bash
#SBATCH --job-name=trinity
#SBATCH -N 1
#SBATCH -t 27-24:00
#SBATCH --partition=himem
#SBATCH --mem=500G
#SBATCH --mail-type=FAIL,END
#SBATCH --mail-user=loe8@txstate.edu
#SBATCH -o trinity_%j.out
#SBATCH -e trinity_%j.err

Trinity --normalize_reads --seqType fa --grid_node_CPU 21 --grid_node_max_memory 300G --max_memory 300G \
--left ../raw_data/concatenated/unmatched_reads1.fa --right ../raw_data/concatenated/unmatched_reads2.fa  \
--CPU 25 --output trinity_out_dir

<a id='5'></a>
### 5. Assembly quality assessment
Before filtering, we did a quality assessment of the raw assembly. The quality assessment consisted of counting the number of transcripts, calculating the N50 of the assembly, and assessing the completeness of the assembly using BUSCO. The first command is a simple `grep` command to count the number of transcripts in the raw assembly file.

In [ ]:
grep -c ">" Trinity.fasta

The total transcript count in the raw assembly is 973,377.

The next step in quality assessment was to calculate the N50 of the assembly. The N50 is a metric that tells you the length of the transcript at the middle of the assembly when transcripts are laid in order from shortest to longest. Trinity provides a script for calculating the N50 in the code from their GitHub repository. If you have not downloaded the Trinity GitHub repository, you will need to do that to run this code.

In [ ]:
~/trinityrnaseq-master/util/TrinityStats.pl Trinity.fasta

The output from the above command is as follows:


`################################

##Counts of transcripts, etc.

################################

Total trinity 'genes':	818951

Total trinity transcripts:	973377

Percent GC: 41.28

########################################

Stats based on ALL transcript contigs:

########################################

	Contig N10: 1641
	Contig N20: 1071
	Contig N30: 732
	Contig N40: 518
	Contig N50: 389

	Median contig length: 267
	Average contig: 394.19
	Total assembled bases: 383692727


#####################################################

##Stats based on ONLY LONGEST ISOFORM per 'GENE':

#####################################################

	Contig N10: 1160
	Contig N20: 675
	Contig N30: 466
	Contig N40: 363
	Contig N50: 298

	Median contig length: 261
	Average contig: 338.47
	Total assembled bases: 277191183`
    
This output shows that the N50 of the overall assembly is 389, and when assessing only the longest isoforms of each transcript the N50 is 298. Ideally the N50 should be >1000 for a good quality assembly.

The last step in the quality assessment is to assess completeness of the assembly using BUSCO. BUSCO stands for Benchmarking Universal Single-Copy Orthologs and compares the transcripts present in the assembly to a database of single copy orthologs for a wide range of taxonomic groups. In this case sequences were compared against the eukaryotic database. The following script was submitted as a job on the LEAP2 server at Texas State.

In [ ]:
#!/bin/bash
#SBATCH --job-name=busco
#SBATCH -N 1
#SBATCH -t 6-24:00
#SBATCH --partition=himem
#SBATCH --mem=250G
#SBATCH --mail-type=end
#SBATCH --mail-user=eborbee@txstate.edu
#SBATCH -o busco_%j.out
#SBATCH -e busco_%j.err

busco -i Trinity.fasta  -l ~/AstrangiaTranscriptome_042622/busco_downloads/eukaryota_odb10 \
-o busco_output -m transcriptome --offline

The BUSCO output for the raw assembly was as follows:

|Category              |Number of BUSCOS|Percentage|
-----------------------|----------------|----------|
|Complete              |182             |71.3%     |
|Complete (single-copy)|46              |18.0%     |
|Complete (duplicated) |136             |53.3%     |
|Fragmented            |52              |20.4%     |
|Missing               |21              |8.3%      |

<a id='6'></a>
### 5. Extracting coding sequences
Next we filtered our transcriptome to coding sequences by extracting the longest open reading frame from each contig using transdecoder. In the same transdecoder script, we generated a file of predicted peptides for each sequence. The code was submitted as a job on the LEAP2 server at Texas State University in the format provided below. 

In [ ]:
#!/bin/bash
#SBATCH --job-name=transDecoder
#SBATCH -N 1
#SBATCH -t 27-24:00
#SBATCH --partition=himem
#SBATCH --mem=500G
#SBATCH --mail-type=end
#SBATCH --mail-user=loe8@txstate.edu
#SBATCH -o transDecoder_%j.out
#SBATCH -e transDecoder_%j.err

TransDecoder.LongOrfs -t Trinity.fasta
TransDecoder.Predict -t Trinity.fasta

The outputs of the script were `Trinity.fasta.transdecoder.cds` containing all of the coding sequences and `Trinity.fasta.transdcoder.pep` containing the predicted peptide sequences. By extracting the coding sequences, we reduced the number of transcripts in the assembly from 973,377 to 218,374.

BUSCO was run on the coding sequence file to assess completeness after filtering. The BUSCO code and outputs are provided below.

In [ ]:
#!/bin/bash
#SBATCH --job-name=busco
#SBATCH -N 1
#SBATCH -t 6-24:00
#SBATCH --partition=himem
#SBATCH --mem=250G
#SBATCH --mail-type=end
#SBATCH --mail-user=eborbee@txstate.edu
#SBATCH -o busco_%j.out
#SBATCH -e busco_%j.err

busco -i Trinity.fasta.transdecoder.cds  -l ~/AstrangiaTranscriptome_042622/busco_downloads/eukaryota_odb10 \
-o busco_cds_output -m transcriptome --offline

BUSCO output for the coding sequences:

|Category              |Number of BUSCOS|Percentage|
-----------------------|----------------|----------|
|Complete              |180             |70.6%     |
|Complete (single-copy)|89              |34.9%     |
|Complete (duplicated) |91              |35.7%     |
|Fragmented            |51              |20.0%     |
|Missing               |24              |9.4%      |

The N50 of the coding sequence file is as follows:

`################################

##Counts of transcripts, etc.

################################

Total trinity 'genes':	151123

Total trinity transcripts:	218374

Percent GC: 44.34

########################################

Stats based on ALL transcript contigs:

########################################

	Contig N10: 1527
	Contig N20: 1134
	Contig N30: 900
	Contig N40: 729
	Contig N50: 603

	Median contig length: 438
	Average contig: 570.98
	Total assembled bases: 124686990


#####################################################

##Stats based on ONLY LONGEST ISOFORM per 'GENE':

#####################################################

	Contig N10: 1323
	Contig N20: 957
	Contig N30: 744
	Contig N40: 606
	Contig N50: 510

	Median contig length: 405
	Average contig: 515.90
	Total assembled bases: 77964321"

<a id='6'></a>
### 5. Clustering sequences by similarity
After extracting coding sequences, we clustered the sequences by similarity using cd-hit to further reduce the size of our transcriptome. The cd-hit code was run on the LEAP2 server at Texas State University using the following script. In this script `-c` represents the sequence identity threshold, `-G` tells the program to either sort the sequence into the first cluster that it meets the threshold for (0) or the most similar cluster (1), `-aL` represents the alignment coverage for the longer sequence, and `-aS` represents the alignment coverage for the shorter sequence.

In [ ]:
#!/bin/bash
#SBATCH --job-name=cdhit
#SBATCH -N 1
#SBATCH -t 6-24:00
#SBATCH --partition=himem
#SBATCH --mem=500G
#SBATCH --mail-type=FAIL,END
#SBATCH --mail-user=loe8@txstate.edu
#SBATCH -o trinity_%j.out
#SBATCH -e trinity_%j.err

cd-hit-est -i Trinity.fasta.transdecoder.cds -o Trinity.transdecoder.cdhit.fasta -c 0.99 -G 0 -aL 0.3 -aS 0.3

The cd-hit step reduced the transcriptome from 218,374 transcripts to 180,914 transcripts. 